- https://www.alibabacloud.com/blog/ai-%ED%98%81%EB%AA%85%EC%9D%98-%EC%A0%90%ED%99%94---rag-%EB%B0%8F-langchain%EA%B3%BC%EC%9D%98-%EC%97%AC%EC%A0%95_601036

In [ ]:
from torch import cuda
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
# from dotenv import load_dotenv

# load_dotenv()

In [ ]:
HUGGINGFACE_API_KEY = '' # key

In [ ]:
modelPath = "sentence-transformers/all-mpnet-base-v2"
device = 'cuda' if cuda.is_available() else 'cpu'
model_kwargs = {'device': device}

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
)

# Made up data, just for fun, but who knows in a future
vectorstore = FAISS.from_texts(
    ["Harrison worked at Alibaba Cloud"], embedding=embeddings
)

retriever = vectorstore.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# This can be changed to any of the Qwen models based on your needs and resources
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1.8B", trust_remote_code=True)
# 7B
model_name_or_path = "Qwen/Qwen-1.8B"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=8192,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | hf
    | StrOutputParser()
)

In [ ]:
results = chain.invoke("Where did Harrison work?")